# ***Dynamic Pricing for Urban Parking Lots***
***Capstone Project of Summer Analytics 2025***


In this notebook implementation of the three pricing models along with their visualisation using Bokeh plots has been done. Using the sample notebook provided to process live data streams using Pathway, this model has been extended to cover all the parking spots. An effective and advanced pricing model has been created for more efficient dynamic pricing.

Below follows the well-commented code based on my interpretation of the project.


In [1]:
!pip install pathway bokeh panel --quiet

In [2]:
#IMPORTING REQD LIBRARIES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [3]:
#IMPORTING DATASET CSV
from google.colab import files
import io
uploaded = files.upload()

print("Uploaded files:", uploaded.keys())
filename = list(uploaded.keys())[0]

df = pd.read_csv(io.BytesIO(uploaded[filename]))
df.head()





Saving dataset.csv to dataset (9).csv
Uploaded files: dict_keys(['dataset (9).csv'])


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00


# Preprocessing the Data

In [4]:
# Combining 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

cleaned = df[[
    'ID','SystemCodeNumber', 'Timestamp', 'Occupancy', 'Capacity', 'QueueLength',
    'TrafficConditionNearby', 'VehicleType', 'IsSpecialDay'
]].rename(columns={
    'TrafficConditionNearby': 'Traffic',
    'IsSpecialDay': 'SpecialDay'
})

unique_ids = cleaned['ID'].unique()
np.random.seed(42)
coords = {
    lot: (25.25 + np.random.rand() / 10, 55.3 + np.random.rand() / 10) for lot in unique_ids
}
cleaned['Latitude'] = cleaned['ID'].map(lambda x: coords[x][0])
cleaned['Longitude'] = cleaned['ID'].map(lambda x: coords[x][1])

cleaned.to_csv("parking_stream.csv", index=False)

In [5]:
import pathway as pw
import datetime

#DEFINING SCHEME FOR STREAMING DATA USING PATHWAY
#SPECIFIES EXPECTED STRUCTURE OF DATA IN STREAM
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str #TIMESTAMP OF OBSERVATION
    Occupancy: int #NUMBER OF OCCUPIED SPOTS
    Capacity: int #TOTAL PARKING CAPACITY AT LOCATION
    QueueLength: int
    Traffic: str
    VehicleType: str
    SpecialDay: bool
    Latitude: float
    Longitude: float

# LOADS DATA AS SIMULATED USING PATHWAY'S replay_csv FUNCTION
#REPLAYS CSV DATA AT CONTROLLED INPUT RATE TO MIMIC REAL-TIME STREAMING
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

#DATETIME FORMAT TO PARSE "TIMESTAMP" COL
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)
#ADDING NEW COL TO DATA STREAM
with open("parking_stream.csv", "r") as f:
    header_line = f.readline()
print(header_line)


ID,SystemCodeNumber,Timestamp,Occupancy,Capacity,QueueLength,Traffic,VehicleType,SpecialDay,Latitude,Longitude



# Making pricing models: Starting off with sample model then expanding into the three models

In [6]:
import pathway as pw
import datetime

# Define schema with SystemCodeNumber as string
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    Traffic: str
    VehicleType: str
    SpecialDay: bool
    Latitude: float
    Longitude: float

# Load streaming data from CSV
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

# Parse timestamp and create 'hour' column (floor to hour)
fmt = "%Y-%m-%d %H:%M:%S"
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
)
data_with_time = data_with_time.with_columns(
    hour = data_with_time.t.dt.floor("1h")
)

# Aggregate hourly per SystemCodeNumber (lot)
hourly = (
    data_with_time
    .groupby(data_with_time.hour, data_with_time.SystemCodeNumber)  # group by hour and lot
    .reduce(
        hour = pw.this.hour,
        lot = pw.this.SystemCodeNumber,
        lat=pw.reducers.max(pw.this.Latitude),
        lon=pw.reducers.max(pw.this.Longitude),
        occ_sum=pw.reducers.sum(pw.this.Occupancy),
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        queue=pw.reducers.max(pw.this.QueueLength),
        cap=pw.reducers.max(pw.this.Capacity),
        traffic=pw.reducers.max(pw.this.Traffic),
        special=pw.reducers.max(pw.this.SpecialDay),
        vehicle=pw.reducers.max(pw.this.VehicleType),
    )
    .with_columns(
        occ_rate=pw.this.occ_sum / pw.this.cap,
        price_model_0=10 + (pw.this.occ_max - pw.this.occ_min) / pw.this.cap,
        t=pw.this.hour,
        lot = pw.this.lot,

    )
)

# Save output CSV and run
pw.io.csv.write(hourly, "hourly_output.csv")
pw.run()


/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


Output()

In [ ]:
# #THIS BLOCK IMPLEMENTS THE THREE DIFFERENT PRICING MODELS AND TRIES TO SEEK A COMPARISON BETWEEN THEM AS ELABORATED IN THE REPORT

import numpy as np
import pandas as pd
from math import radians, sin, cos, sqrt, atan2
from scipy.spatial import cKDTree
import numpy as np

df = pd.read_csv("hourly_output.csv")
df['t'] = pd.to_datetime(df['t'])

# MODEL 1: Recursive
ALPHA = 2.0
price1 = [10]
for occ in df['occ_rate'].iloc[1:]:
    price1.append(price1[-1] + ALPHA * occ)
df['price_model_1'] = price1

# MODEL 2: Demand-Based
weights = {'TwoWheeler': 0.5, 'Sedan': 1.0, 'SUV': 1.5, 'Truck': 2.0}
traffic_map = {'Low': 0.2, 'Medium': 0.5, 'High': 1.0}

df['vehicle_wt'] = df['vehicle'].map(weights).fillna(1.0)
df['traffic_num'] = df['traffic'].map(traffic_map).fillna(0.5)

max_queue = df['queue'].max()
if max_queue == 0:
    df['queue_norm'] = 0.0
else:
    df['queue_norm'] = df['queue'] / max_queue

# Normalize occupancy rate to [0,1]
df['occ_rate'] = df['occ_rate'].clip(0,1)

epsilon = 1e-3 #adding small epsilon to avoid zero or negative demand
demand = (
    0.5 * df['occ_rate'] +
    0.3 * df['queue_norm'] -
    0.1 * df['traffic_num'] +
    1.0 * df['special'].astype(float) +
    df['vehicle_wt'] * 0.1
)

# Normalizing demand
demand_min = demand.min()
demand_max = demand.max()
if demand_max - demand_min == 0:
    demand_norm = demand - demand_min
else:
    demand_norm = (demand - demand_min) / (demand_max - demand_min)

base_price = 10
lambda_param = 1.0
price_model_2_raw = base_price * (1 + lambda_param * demand_norm)

df['price_model_2'] = price_model_2_raw.clip(lower=base_price * 0.5, upper=base_price * 2)

# MODEL 3: Competitive Pricing Model
from scipy.spatial import cKDTree
import numpy as np
from math import radians, sin, cos, sqrt, atan2

# Function for haversine distance (used only for reroute message formatting)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

coords_rad = np.radians(df[['lat', 'lon']].values)
tree = cKDTree(coords_rad)

radius_km = 1.0
earth_radius_km = 6371.0
radius_rad = radius_km / earth_radius_km

nearby_indices = tree.query_ball_tree(tree, r=radius_rad)

model3_prices = []
reroute_msgs = []

for i, neighbors in enumerate(nearby_indices):
    base_price = df.at[i, 'price_model_2']
    occ1 = df.at[i, 'occ_rate']
    lot1 = df.at[i, 'lot']
    lat1, lon1 = df.at[i, 'lat'], df.at[i, 'lon']

    # Excluding self
    neighbors = [j for j in neighbors if j != i]

    cheaper = [j for j in neighbors if df.at[j, 'price_model_2'] < base_price]
    expensive = [j for j in neighbors if df.at[j, 'price_model_2'] > base_price]

    if occ1 > 0.9 and cheaper:
        adjusted_price = max(base_price - 0.5, base_price * 0.5)
        dist = haversine(lat1, lon1, df.at[cheaper[0], 'lat'], df.at[cheaper[0], 'lon'])
        reroute_msgs.append(
            f"Lot {lot1} is full. Suggest Lot {df.at[cheaper[0], 'lot']} ({dist:.2f} km away)"
        )
    elif expensive:
        adjusted_price = min(base_price + 0.3, base_price * 2)
        reroute_msgs.append("")
    else:
        adjusted_price = base_price
        reroute_msgs.append("")

    model3_prices.append(adjusted_price)

df['price_model_3'] = model3_prices
df['reroute_msg'] = reroute_msgs

df[['price_model_2', 'price_model_3']] = df[['price_model_2', 'price_model_3']].fillna(method='ffill').fillna(method='bfill')




In [ ]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
import panel as pn

pn.extension('bokeh')

df = pd.read_csv("hourly_output.csv")
df['t'] = pd.to_datetime(df['t'])

for col in ['price_model_0', 'price_model_1', 'price_model_2', 'price_model_3']:
    if col not in df.columns:
        df[col] = 0.0  # default zero
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

df['lot'] = pd.to_numeric(df['lot'], errors='coerce').fillna(-1).astype(int)

tabs = []

unique_lots = sorted(df['lot'].unique())

for lot in unique_lots:
    lot_df = df[df['lot'] == lot].copy()
    lot_df = lot_df.sort_values('t').reset_index(drop=True)

    print(f"Plotting Lot {lot} with {len(lot_df)} rows")

    if lot_df.empty:
        print(f"Lot {lot} has no data — skipping")
        continue

    source = ColumnDataSource(data=dict(
        t=lot_df['t'],
        model0=lot_df['price_model_0'],
        model1=lot_df['price_model_1'],
        model2=lot_df['price_model_2'],
        model3=lot_df['price_model_3'],
    ))

    p = figure(
        title=f"Dynamic Pricing Models for Lot {lot}",
        x_axis_type='datetime',
        width=900,
        height=400,
        tools="pan,wheel_zoom,box_zoom,reset,save"
    )
    p.line('t', 'model0', source=source, legend_label="Model 0", line_dash='dashed', color='gray')
    p.line('t', 'model1', source=source, legend_label="Model 1", color='blue')
    p.line('t', 'model2', source=source, legend_label="Model 2", color='orange')
    p.line('t', 'model3', source=source, legend_label="Model 3", color='green', line_width=2)

    p.legend.location = "top_left"
    p.legend.click_policy = "hide"
    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Price ($)"

    tabs.append(pn.panel(p))

tabs_layout = pn.Tabs(*tabs)
tabs_layout


In [10]:
#FOR REROUTING SUGGESTIONS
for msg in reroute_msgs[:10]:
    print("Rerouting Message:", msg)


Rerouting Message: Lot 17 is full. Suggest Lot 35 (0.18 km)
Rerouting Message: Lot 35 is full. Suggest Lot 125 (0.21 km)
Rerouting Message: Lot 53 is full. Suggest Lot 107 (0.86 km)
Rerouting Message: Lot 71 is full. Suggest Lot 35 (0.17 km)
Rerouting Message: Lot 89 is full. Suggest Lot 53 (0.89 km)
Rerouting Message: Lot 107 is full. Suggest Lot 125 (0.87 km)
Rerouting Message: Lot 125 is full. Suggest Lot 233 (0.25 km)
Rerouting Message: Lot 143 is full. Suggest Lot 35 (0.37 km)
Rerouting Message: Lot 161 is full. Suggest Lot 449 (0.41 km)
Rerouting Message: Lot 179 is full. Suggest Lot 53 (0.67 km)
